In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection

from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential

##Importamos las matrices a utilizar

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
id= '1yFTqxomrfb9xZmOck780mcr7Fb9Y9R5S'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_test_use_solo_text.csv')

id= '1knojPsOX9jd1LewsNBPd9EYbqBPwVyFr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_train_use_solo_text.csv')

In [5]:
matriz_test_use_solo_text = pd.read_csv('matriz_test_use_solo_text.csv')
matriz_train_use_solo_text = pd.read_csv('matriz_train_use_solo_text.csv')

##Split del set de datos


In [6]:
x_train_text, x_test_text, y_train_text, y_test_text = train_test_split(matriz_train_use_solo_text, train.target, test_size=0.2, random_state = 1)

In [7]:
def cross_val(model, x_train, y_train):
  score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
  print(score_cross_val.mean())

#NuSVC

In [8]:
def nusvc(x_train, y_train, x_validation, y_validation):

  nusvc = NuSVC(probability=True)
  grid_nusvc = GridSearchCV(nusvc, param_grid={'nu': [0.4, 0.5]}, cv=5)
  grid_nusvc.fit(x_train, y_train)
  grid_best = grid_nusvc.best_estimator_
  
  print(grid_nusvc.best_params_)
  print('nusvc score: {}'.format(grid_best.score(x_validation, y_validation)))

  return grid_best

In [9]:
nusvc_text = nusvc(x_train_text, y_train_text, x_test_text, y_test_text)

{'nu': 0.4}
nusvc score: 0.8397898883782009


In [11]:
prediction_nuscv = nusvc_text.predict(matriz_test_use_solo_text)
submission_nuscv = pd.DataFrame(data={'id':test['id'], 'target': prediction_nuscv})
submission_nuscv.to_csv('hub_nuscv_text.csv', index=False)